In [ ]:
import random
from datasets import load_dataset
from datasets import DatasetDict
import re
from transformers import AutoTokenizer

In [ ]:
dataset = load_dataset("jerryjalapeno/nart-100k-synthetic")
print(dataset)

In [ ]:
# choose 30000 samples from the dataset
# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# print(dataset)

In [ ]:
# create a train_test_val split
train_test_split = dataset["train"].train_test_split(test_size=0.3)
train_val_split = train_test_split['test'].train_test_split(test_size=0.5)

print(train_test_split)
print(train_val_split)

train_test_val_split = DatasetDict({
    "train": train_test_split['train'],
    "test": train_val_split['train'],
    "val": train_val_split['test']
})

print(train_test_val_split)


In [ ]:
SYSTEM_PROMPT = """You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def preprocessText(text):
    text = re.sub(r'Alex', '', text)
    text = re.sub(r'Charlie', '', text)
    # remove ", " when it appears at the start of a sentence
    text = re.sub(r'^, ', '', text)
    # remove " ." with "."
    text = re.sub(r' \.', '.', text)
    # remove " ," with ","
    text = re.sub(r' ,', ',', text)
    # remove " ?" with "?"
    text = re.sub(r' \?', '?', text)
    # remove ",." with "."
    text = re.sub(r',\.', '.', text)
    # remove ",?" with "?"
    text = re.sub(r',\?', '?', text)
    # remove more than one space
    text = re.sub(r' +', ' ', text)
    
    return text
    

def transform_dataset(data_row):
    id = data_row['id']
    data_row = data_row['conversations']
    for conv in data_row:
        if conv['from'] == 'human':
            conv['role'] = "user"
        elif conv['from'] == 'gpt':
            conv['role'] = "assistant"
        
        conv['content'] = preprocessText(conv['value'])
        del conv['from']
        del conv['value']
    
    system_dict = {}
    system_dict['role'] = "system"
    system_dict['content'] = SYSTEM_PROMPT
    data_row.insert(0, system_dict)
    # in conversational format the features name is "messages"
    return {"messages": data_row}

train_test_val_split = train_test_val_split.map(transform_dataset, remove_columns=['conversations'])


In [ ]:
print(train_test_val_split)

In [ ]:
print(train_test_val_split['train'][0]['messages'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("llama-2-7b-chat-hf-phr_mental_therapy-2")
print(tokenizer.apply_chat_template(train_test_val_split['train'][0]['messages'],tokenize=False))

In [ ]:
train_test_val_split.push_to_hub("phr-mental-therapy-dataset-conversational-format")

In [ ]:
### Get the max sequence length of the tokenized messages                                          
train_test_val_split = train_test_val_split.map(lambda x: {"input_ids": tokenizer.apply_chat_template(x["messages"])})
print(train_test_val_split["train"][0]["input_ids"] )
# get the maximum length of the formatted chat
max_length = max(len(chat) for chat in train_test_val_split["train"]["input_ids"])
print(max_length)